# To generate cropped tiff from big Tiff and Geojson
### Virtual Environment: remote_sensing_v2

In [1]:
import os
os.chdir('/home/GIS_WS/GIS_Roads_WS')

In [2]:
home_direc = os.getcwd()
print(home_direc)

/home/GIS_WS/GIS_Roads_WS


##### Dictionaries for Cropping the tiff and geojson
### Check for the cropping size and overlap rate

In [4]:
# crop and convert the geojson and tiff file to coco format dataset

import os
import rasterio
from osgeo import gdal
import numpy as np
from tqdm.auto import tqdm, trange
import geopandas as gpd
from osgeo import ogr
from shapely.geometry import Polygon
import matplotlib.pyplot as plt
import cv2

# Save the raster image
def writeTiff(im_data, save_path, new_transform, crs):
    _, height, width = im_data.shape
    with rasterio.open(save_path,
                       'w',
                       driver='GTiff',
                       height=height,
                       width=width,
                       count=3,
                       dtype=im_data.dtype,
                       crs=crs,
                       transform=new_transform
                       ) as dst:
        dst.write(im_data)
    return

# read the raster image
def readTiff(TifPath):
    dataset_img = rasterio.open(TifPath)
    width = dataset_img.width
    height = dataset_img.height
    crs = dataset_img.crs
    transform = dataset_img.transform # (xres, 0, xcord, 0, yres, ycord)
    img_array = dataset_img.read([1, 2, 3]) # get the data
    return img_array, width, height, crs, transform

# Crop tiff image by sliding window method
def crop_tif(img_path, save_dir, stride=640, overlap_rate=0.20):
    # Side length of the overlapping area
    step = int(stride * overlap_rate)
    im_data, width, height, crs, transform = readTiff(img_path)
    im_name, _ = os.path.splitext(os.path.split(img_path)[-1])
    x_res = transform[0]
    y_res = transform[4]
    x_cord = transform[2]
    y_cord = transform[5]
    # Calculate the coordinates of the cropped image vertices for each row and column
    w_step = [i for i in range(0, width - stride, stride - step)]
    w_step.append(width - stride)
    h_step = [i for i in range(0, height - stride, stride - step)]
    h_step.append(height - stride)
    
    for i in trange(len(h_step)): # Crop row
        for j in range(len(w_step)): # Crop column
            cropped = im_data[:, h_step[i] : h_step[i] + stride, w_step[j] : w_step[j] + stride]
            # Calculate coordinate
            new_x_cord = x_cord + w_step[j] * x_res  
            new_y_cord = y_cord + h_step[i] * y_res  
            new_transform = (x_res, transform[1], new_x_cord, transform[3], y_res, new_y_cord)
            # save the image
            save_name = '{}_{}_{}.tif'.format(im_name, i, j) # Name of top left figure is 0_0.tif
            writeTiff(cropped, os.path.join(save_dir, save_name), new_transform, crs)
    print('Cropping TIFF finished!')


# crop the geojson acoording to the extent of geotiff
def crop_geojson(geojson_path, tif_dir, save_dir):
    # read geojson
    annot_df = gpd.read_file(geojson_path)
    # get the list of tiff files
    tif_list = [x for x in os.listdir(tif_dir) if x.endswith('tif')]
    tif_list.sort()
    for tif in tqdm(tif_list):
        tif_path = os.path.join(tif_dir, tif)
        # read tif and get the extent
        img_dataset = rasterio.open(tif_path)
        img_name = os.path.splitext(os.path.split(tif_path)[-1])[0]
        left, bottom, right, top = img_dataset.bounds
        # create mask in geopandas (minx, miny, maxx, maxy)
        mask = gpd.GeoSeries(Polygon([(left, bottom), (left, top), (right, top), (right, bottom)]), crs = annot_df.crs)
        mask_clip = gpd.clip(annot_df, mask)
        # Save geojson if cropped data is not empty
        if mask_clip.shape[0] != 0:
            # save path
            save_path = os.path.join(save_dir, img_name + '.geojson')
            mask_clip.to_file(save_path, driver= 'GeoJSON')
    print('Cropping GEOJSON finished!')

# translate Gdal data to opencv format
def GdalData2OpencvData(GdalImg_data):
    OpencvImg_data = np.zeros((GdalImg_data.shape[1],GdalImg_data.shape[2],GdalImg_data.shape[0]),np.uint8)
    for i in range(GdalImg_data.shape[0]):
        OpencvImg_data[:,:,i] = GdalImg_data[GdalImg_data.shape[0]-i-1,:,:]
    return OpencvImg_data

ModuleNotFoundError: No module named 'osgeo'

##### Cropping the TIFF image to small patches

In [9]:
# crop the tiff and geojson to patches

#root_direc = 'scene_classification_ws'  # path of root diretory
tif_path = os.path.join(home_direc, 'ortho.tif') # path of tiff images
save_crop_tiff_dir = os.path.join(home_direc,  'cropped_tiff')  # path to cropped geojson and tiff files
os.makedirs(save_crop_tiff_dir, exist_ok=True)

# crop the tif to size of 1024*1024, if you need
crop_tif(tif_path, save_crop_tiff_dir)

  0%|          | 0/27 [00:00<?, ?it/s]

Cropping TIFF finished!


In [10]:
from PIL import Image
import os

# Path to the folder containing TIFF images
tiff_folder = 'cropped_tiff'

# Path to the folder where you want to save the converted JPG images
jpg_folder = 'cropped_jpg'

# Create the output folder if it doesn't exist
os.makedirs(jpg_folder, exist_ok=True)

# Iterate through all TIFF files in the folder
for filename in os.listdir(tiff_folder):
    if filename.endswith('.tif') or filename.endswith('.tiff'):
        # Full path to the input TIFF file
        tiff_path = os.path.join(tiff_folder, filename)

        # Output path for the converted JPG file
        jpg_path = os.path.join(jpg_folder, os.path.splitext(filename)[0] + '.jpg')

        # Open the TIFF image
        tiff_image = Image.open(tiff_path)

        # Save the TIFF image as a JPG image
        tiff_image.save(jpg_path, 'JPEG')

        print(f"Conversion complete: {filename} -> {os.path.basename(jpg_path)}")


Conversion complete: ortho_11_9.tif -> ortho_11_9.jpg
Conversion complete: ortho_16_22.tif -> ortho_16_22.jpg
Conversion complete: ortho_2_12.tif -> ortho_2_12.jpg
Conversion complete: ortho_24_14.tif -> ortho_24_14.jpg
Conversion complete: ortho_19_12.tif -> ortho_19_12.jpg
Conversion complete: ortho_23_19.tif -> ortho_23_19.jpg
Conversion complete: ortho_5_21.tif -> ortho_5_21.jpg
Conversion complete: ortho_14_22.tif -> ortho_14_22.jpg
Conversion complete: ortho_9_23.tif -> ortho_9_23.jpg
Conversion complete: ortho_0_13.tif -> ortho_0_13.jpg
Conversion complete: ortho_0_23.tif -> ortho_0_23.jpg
Conversion complete: ortho_21_1.tif -> ortho_21_1.jpg
Conversion complete: ortho_7_23.tif -> ortho_7_23.jpg
Conversion complete: ortho_7_8.tif -> ortho_7_8.jpg
Conversion complete: ortho_1_23.tif -> ortho_1_23.jpg
Conversion complete: ortho_1_14.tif -> ortho_1_14.jpg
Conversion complete: ortho_22_15.tif -> ortho_22_15.jpg
Conversion complete: ortho_13_6.tif -> ortho_13_6.jpg
Conversion complet